In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.model_selection import validation_curve

In [ ]:
hp=pd.read_csv('home_price.csv')
hp.columns

In [ ]:
import statsmodels.formula.api as smf
lm1 = smf.ols(formula='price ~  bedrooms + bathrooms + sqft_living+ sqft_lot+ floors +waterfront+view +condition+ grade+sqft_above+sqft_basement+yr_built+ yr_renovated+ zipcode+lat+long+sqft_living15+ sqft_lot15', data=hp).fit()

In [ ]:
#run a significant test to select features
lm1.summary()

In [ ]:
# casue p-value for floors is larger than 0.05, delete the floors
feature_house1 = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15']
X1_house = hp[feature_house1]
y_house = hp['price']
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_house, y_house,random_state = 42)
linreg = LinearRegression().fit(X1_train, y1_train)
print('linear model intercept: {}'
     .format(linreg.intercept_))
print('linear model coeff:\n{}'
     .format(linreg.coef_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X1_train, y1_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X1_test, y1_test)))

In [ ]:
#function that return different combinations of 'bedrooms' and other features(except 'id', 'date', 'price' and 'bedrooms') 
y=hp['price']
iv=hp[hp.columns[3:]]

def iv(x):
    if x in range(18):
        return hp[hp.columns[3:21-x]]
    else:
        print('wrong number')

In [ ]:
#let price be y and the combinations from In[3] be X, show the coefficients and intercept for different linear models
##since deleting floor does not improve the model, we keep it anyway
for i in range(18):
    X_train, X_test, y_train, y_test = train_test_split(iv(i), y,random_state=42)
    linreg = LinearRegression().fit(X_train, y_train)
    print('linear model coeff (w): {}'.format(linreg.coef_))
    print('linear model intercept (b): {:.3f}'.format(linreg.intercept_))
    print('R-squared score (training): {:.3f}'.format(linreg.score(X_train, y_train)))
    print('R-squared score (test): {:.3f}'.format(linreg.score(X_test, y_test)))

In [ ]:
#function that return ssr for different model using different input features
def ssr(i):
    if i in range(18):
        X_train, X_test, y_train, y_test = train_test_split(iv(i),y,random_state=42)
        lr=LinearRegression().fit(X_train, y_train)
        y_predict=lr.predict(X_test)
        ssr=sum((y_test - y_predict)**2)
        return ssr

X_train, X_test, y_train, y_test = train_test_split(iv(17),y,random_state=42)
lr=LinearRegression().fit(X_train, y_train)
yp=lr.predict(X_test)


plt.plot(iv(17),lr.coef_ *iv(17)+lr.intercept_, 'r-')
plt.title('linear regression')
plt.xlabel('Feature value (x)')
plt.ylabel('Target value (y)')
plt.show()

In [ ]:
#print the ssr for all different input features
#it seems in this case that the input with all variables except 'id', 'date', 'price' has the best ssr result
for i in range(18):
    print('ssr=',ssr(i))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import validation_curve

In [ ]:
#visualize polynomial regressions for each and every input feature(one feature each time) with degree = 3
for i in range(3,18):
    X_train, X_test, y_train, y_test = train_test_split(hp[hp.columns[i]],y,random_state=42)
    coeffs=np.polyfit(X_train,y_train,3)
    x2=np.arange(min(hp[hp.columns[i]])-1, max(hp[hp.columns[i]])+1, .01) #use more points for a smoother plot
    y2=np.polyval(coeffs, x2) #Evaluates the polynomial for each x2 value
    plt.plot(x2,y2)
    plt.scatter(hp[hp.columns[i]], y,marker='o')
    plt.show()

In [ ]:
from sklearn.pipeline import make_pipeline

#function that combines poly and lr
def PolynomialRegression(degree=i, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))


In [ ]:
#let the parameter be degree in range of (1,5)
#let X be different input features(one feature each time)
#function that returns the train scores and test scores of 3 polynomial regression models using different subset of the data 
i_range=range(1,5)
def fpoly(cl):
    if cl in range(3,18):
        X=hp[hp.columns[cl]]
        X_train, X_test, y_train, y_test = train_test_split(X.values.reshape(-1,1),y,random_state=42)
        for i in i_range:
            train_scores, test_scores = validation_curve(PolynomialRegression(degree=i).fit(X_train,y_train),X.values.reshape(-1,1),y,'polynomialfeatures__degree',i_range)
            return train_scores, test_scores

In [ ]:
#print train score and test score for different input features and degree = 1,2,3,4
#it seems that result for input feature 'sqft_living' with degree=3 is the best
for i in range(3,18):
    print(fpoly(i))

In [ ]:
i_range=range(1,5)
X_train5, X_test5, y_train5, y_test5 = train_test_split(hp[hp.columns[5]].values.reshape(-1,1),y,random_state=42)
train_scores5, test_scores5 = validation_curve(PolynomialRegression(degree=i).fit(X_train5,y_train5),hp[hp.columns[5]].values.reshape(-1,1),y,'polynomialfeatures__degree',i_range)
           

In [ ]:
test_scores5

In [ ]:
#use input feature = 'sqft_living' 
#plot the mean train and test scores across the 3 models using different subsets of the data for different degre
#it seems that degree=3 has the best result

plt.figure()
train_scores_mean = np.mean(train_scores5, axis=1)
test_scores_mean = np.mean(test_scores5, axis=1)
plt.xlabel('$\degree$ (degree)')
plt.ylabel('Score')
plt.ylim(0.4,0.65)

lw=3
plt.semilogx(i_range, train_scores_mean, label='Train score',
            color='green',lw=lw)
plt.semilogx(i_range, test_scores_mean, label='Test score',
            color='red',lw=lw)
plt.legend(loc='best')
plt.show()

In [ ]:
#assess the final fit using test data
Final_P=PolynomialRegression(degree=3)
Final_P.fit(X_train5,y_train5)
predicted_price=Final_P.predict(X_test5)
trains=Final_P.score(X_train5,y_train5)
tests=Final_P.score(X_test5,y_test5)
trains,tests

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#use a pre-built implementation of regression to run polynomial regression and visualize polynomial regressions
X_train, X_test, y_train, y_test = train_test_split(hp[hp.columns[5]],y,random_state=42)
coeffs=np.polyfit(X_train,y_train,3)
x3=np.arange(min(hp[hp.columns[5]])-1, max(hp[hp.columns[5]])+1, .01) 
y3=np.polyval(coeffs, x3) 
plt.plot(x3,y3)
plt.scatter(hp[hp.columns[5]], y,marker='o')
plt.show()

In [ ]:
#scale the data so that the result for L1 and L2 will be better
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

poly = PolynomialFeatures(degree=3)
X5_poly = poly.fit_transform(hp[hp.columns[5]].values.reshape(-1,1))
X5_train, X5_test, y5_train, y5_test = train_test_split(X5_poly, y,random_state=42)
X5_train_scaled=scaler.fit_transform(X5_train)
X5_test_scaled=scaler.transform(X5_test)
X5_scaled=scaler.transform(X5_poly)

In [ ]:
#apply L2 to the model, it seems that alpha should be 10 or 20
for i in [0,1,10,20,50,100,1000]:
    linridge=Ridge(alpha=i)
    linridge.fit(X5_train_scaled, y5_train)
    train1=linridge.score(X5_train_scaled, y5_train)
    test1=linridge.score(X5_test_scaled, y5_test)
    print(train1,test1)

In [ ]:
#use validation to prove that the best penalty for L2 should be alpha=10
l=[0,1,10,20,50,100,500,1000]
for j in l:
    train_scoresL2, test_scoresL2 = validation_curve(Ridge(j).fit(X5_train_scaled,y5_train),X5_scaled,y,'alpha',l,cv=10)
trainL2=train_scoresL2.sum(axis=1)/10
testL2=test_scoresL2.sum(axis=1)/10
trainL2,testL2

In [ ]:
#visualization of polynomial regressions under L2 regularization
r=Ridge(alpha=10)
r.fit(X5_train_scaled,y5_train)
poly=PolynomialFeatures(degree=3)
x3p=poly.fit_transform(x3.reshape(-1,1))
x3s=scaler.transform(x3p)
yp=r.predict(x3s)

plt.scatter(hp[hp.columns[5]], y,marker='o',label='actual price')
plt.plot(x3,y3,label='poly',color='orange')
plt.plot(x3,yp,label='poly under L2',color='green')
plt.legend(loc='best')
plt.show()

In [ ]:
#assess the final fit using test data
Final_L2=Ridge(alpha=10)
Final_L2.fit(X5_train_scaled,y5_train)
predicted_priceL2=Final_L2.predict(X5_test_scaled)
trainF2=Final_L2.score(X5_train_scaled,y_train5)
testF2=Final_L2.score(X5_test_scaled,y_test5)
trainF2,testF2

In [ ]:
#apply L1 to the model, split the data to training and testing, it seems that alpha should be 1000
for i in [1,10,20,50,100,1000,1100]:
    linlasso=Lasso(alpha=i, max_iter = 10000).fit(X5_train_scaled, y5_train)
    train2=linlasso.score(X5_train_scaled, y5_train)
    test2=linlasso.score(X5_test_scaled, y5_test)
    print(train2,test2)

In [ ]:
#use validation to show the best penalty, it seems that L1's alpha should be 200
#we will use alpha=200 instead

l=[10,50,100,200,300,400,500,1000]
for j in l:
    train_scoresL1, test_scoresL1 = validation_curve(Lasso(alpha=j),X5_scaled,y,'alpha',l,cv=10)
trainL1=train_scoresL1.sum(axis=1)/10
testL1=test_scoresL1.sum(axis=1)/10
trainL1,testL1

In [ ]:
#assess the final fit using test data
Final_L1=Lasso(alpha=200)
Final_L1.fit(X5_train_scaled,y5_train)
predicted_priceL1=Final_L1.predict(X5_test_scaled)
trainF1=Final_L1.score(X5_train_scaled,y_train5)
testF1=Final_L1.score(X5_test_scaled,y_test5)
trainF1,testF1

In [ ]:
#from the coefficients we can see that the list of significant features should be sqft_living^1 and sqft_living^2
coefs = []
linlasso=Lasso(alpha=200)
linlasso.fit(X5_train_scaled, y5_train)
coefs=linlasso.coef_
coefs

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#choose all features as input in the knn model, use validation curve to find the best n
##this step may take a long time to run, please be patient
from sklearn.neighbors import KNeighborsRegressor

k_range=[1,3,5,10,15,20,25,30]
X_train, X_test, y_train, y_test = train_test_split(iv(1), y, random_state = 42)
for k in k_range:
    train_scores, test_scores = validation_curve(KNeighborsRegressor(n_neighbors=k), iv(1), y, 'n_neighbors', k_range) 

In [ ]:
train_scores_mean=np.mean(train_scores, axis=1)
test_scores_mean=np.mean(test_scores, axis=1)
train_scores_mean,test_scores_mean

In [ ]:
#yellow dots:training scores, blue dots:test scores
#plot the mean score of training and testing for the model(input features:all except id, date and price)
#we pick k=15 as the best n
for k in k_range:
    knnreg = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, train_scores_mean, color='orange')
plt.scatter(k_range, test_scores_mean, color='navy')
plt.xticks([0,5,10,15,20,25,30,35]);
plt.show()

In [ ]:
#use sqft_living as input feature, use validation curve to find the best n
X5= hp[hp.columns[5]].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X5, y, random_state = 42)
train_scores, test_scores = validation_curve(KNeighborsRegressor(n_neighbors = k), X5, y, param_name='n_neighbors', param_range= k_range) 

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_mean,test_scores_mean

In [ ]:
#best n=30
for k in k_range:
    knnreg = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, train_scores_mean, color='orange')
plt.scatter(k_range, test_scores_mean, color='navy')
plt.xticks([0,5,10,15,20,25,30,35]);
plt.show()

In [ ]:
#input feature: grade, use validation curve to find the best n
X11= hp[hp.columns[11]].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X11, y, random_state = 42)
train_scores, test_scores = validation_curve(KNeighborsRegressor(n_neighbors = k), X11, y, param_name='n_neighbors', param_range= k_range) 

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_mean,test_scores_mean

In [ ]:
#best n=25
for k in k_range:
    knnreg = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, train_scores_mean, color='orange')
plt.scatter(k_range, test_scores_mean, color='navy')
plt.xticks([0,5,10,15,20,25,30,35]);
plt.show()

In [ ]:
#input feature: grade and sqft_living, use validation curve to find the best n
Xsg= hp[['sqft_living','grade']]
X_train, X_test, y_train, y_test = train_test_split(Xsg, y, random_state = 42)
train_scores, test_scores = validation_curve(KNeighborsRegressor(n_neighbors = k), Xsg, y, param_name='n_neighbors', param_range= k_range) 

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_mean,test_scores_mean

In [ ]:
#best n=20
for k in k_range:
    knnreg = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, train_scores_mean, color='orange')
plt.scatter(k_range, test_scores_mean, color='navy')
plt.xticks([0,5,10,15,20,25,30,35]);
plt.show()

In [ ]:
#from all these model we can see that the best one is n=15 and input features are all except id, date and price
#predict price using the k-nearest neighbors
X_train, X_test, y_train, y_test = train_test_split(iv(1), y, random_state = 42)
KR=KNeighborsRegressor(n_neighbors=15).fit(X_train,y_train)
predictP=KR.predict(X_test)

In [ ]:
predictP